# Prepare for synthesized Temporal Sequence Datasets

In [ ]:
import numpy as np
import json
import os

from tqdm import tqdm
from collections import Counter

## Rules of version 10

Sequence length T = 20;

Token Types (Token-Encoding-meaning): 

P-0-padding, reserved token

N-1-initial token

A-2-start

B-3-view

C-4-click

D-5-install


In order to be positive, a temporal sequence MUST NOT violate any of the following rules / hidden patterns:

0. __[Increasing time]__: Timestamp must be strictly increasing. A later event must have a greater timestamp then any previous event. Since delta_t is used for data generation, timestamp will always increasing, so __this rule is NOT used for oracle__

1. __[Starting with A]__: A sequence must start with an A event.

2. __[At least 3 types]__: There must be at least 3 distinct types of token after the init token, and at least 1 in the 3 types should be A.

3. __[Pairing C & D]__: Each C event can either appear alone, or be paired with one and only one later D event. Each D event has to be paired with one and only one previous C event. Pairing can be non-unique. 

4. __[Number Decay]__: The total number of A's must be greater than B; The total number of B's must be >= the nums of C; The total number of C's must be >= the nums of D.

5. __[Minimum Same Delay]__: The minimum time delay between two consecutive __same__ tokens is 10 secs

6. __[Maximum Pair Delay]__: The time delay between a paired C and D cannot be > 50 secs

## Timestamp distributions conditioned on the upcoming event

In [ ]:
# the ts distribution is conditioned on the previous event
# e.g. if the upcoming event is an A, it follows chi-square 8 distribution
event_to_ts_dist = dict({
    'A' : lambda: np.random.chisquare(df=8),
    'B' : lambda: np.random.chisquare(df=16),
    'C' : lambda: np.random.chisquare(df=24),
    'D' : lambda: np.random.chisquare(df=32),
})

## Define the Context and Rules

In [ ]:
# EVENT_TYPES = {0:'':'A', 3:'B', 4:'C'} # 0 is reserved for padding 1 is for 'init token'
EVENT_TYPES = ['P', 'N', 'A', 'B', 'C', 'D']
EVENT_ENCODE = {'P':0, 'N':1, 'A':2, 'B':3, 'C':4, 'D':5}
INIT_TOKEN = EVENT_ENCODE['N']
END_TOKEN = EVENT_ENCODE['P']

MIN_SAME_DELAY = 10
MAX_PAIR_DELAY = 50

def check_increasing_rule(seq):
    for i in range(1, len(seq)):
        if seq[i][1] <= seq[i-1][1]:
            return False
    return True


def check_rule_1(seq, use_init_token=True):
    if use_init_token:
        seq = seq[1:]
    return seq[0][0] == EVENT_ENCODE['A']
        
    
def check_rule_2(seq, use_init_token=True):
    if use_init_token:
        seq = seq[1:]
    cnt = Counter()
    for et, dt in seq:
        cnt[et] += 1
    # rule 2
    if len(cnt.keys()) > 3 and EVENT_ENCODE['A'] in cnt.keys():
        return True
    else:
        return False


def check_rule_3(seq, use_init_token=True):
    if use_init_token:
        seq = seq[1:]    
    # one-pass: add D to queue to be attributed to the first available C in a reversed linear scanning
    queue = []
    for i in range(len(seq)-1, -1, -1):
        if seq[i][0] == EVENT_ENCODE['D']: # encounter a D event
            queue.append(i)
        elif seq[i][0] == EVENT_ENCODE['C'] and queue: # encounter a C event
            queue.pop(0)
    return len(queue) == 0


def check_rule_4(seq, use_init_token=True):
    if use_init_token:
        seq = seq[1:]
    cnt = Counter()
    for et, dt in seq:
        cnt[et] += 1
    # rule 4
    if cnt[EVENT_ENCODE['A']] < EVENT_ENCODE['B']:
        return False
    if cnt[EVENT_ENCODE['B']] < EVENT_ENCODE['C']:
        return False
    if cnt[EVENT_ENCODE['C']] < EVENT_ENCODE['D']:
        return False
    return True


def check_rule_5(seq, use_init_token=True):
    if use_init_token:
        seq = seq[1:]
    prev_et, _ = EVENT_ENCODE['N'], 0.0
    for et, dt in seq:
        if et == prev_et and dt < MIN_SAME_DELAY:
            return False
        prev_et = et
    return True


def check_rule_6(seq, use_init_token=True):    
    if use_init_token:
        seq = seq[1:]    
        
    def recover_timedelta_to_timestamp(time_seq):
        csum = []
        curr = 0
        for dt in time_seq:
            if dt != 0:
                curr += dt
                csum.append(curr)
            else:
                csum.append(0)
        return csum
    
    ets = [e[0] for e in seq]
    tss = recover_timedelta_to_timestamp([e[1] for e in seq])
        
    # one-pass: add D to queue to be attributed to the first available C in a reversed linear scanning
    queue = []
    for i in range(len(seq)-1, -1, -1):
        if ets[i] == EVENT_ENCODE['D']: # encounter a D event
            queue.append(i)
        elif ets[i] == EVENT_ENCODE['C'] and queue: # encounter a C event
            if tss[queue[0]] - tss[i] <= MAX_PAIR_DELAY:
                queue.pop(0)
            else:
                return False
    # for rule 6, it's fine if there are unpaired D in queue
    # b/c this rules is to ensure for each paired (C, D), the delay is bounded
    return True


def get_rule_dist(seqs, use_init_token=True):
    seq_to_rules = [0] * 7
    N = len(seqs)
    
    for i in tqdm(range(N)):
        seq = seqs[i]
        # check rules one by one:
        if check_rule_1(seq):
            seq_to_rules[1] += 1
        if check_rule_2(seq):
            seq_to_rules[2] += 1
        if check_rule_3(seq):
            seq_to_rules[3] += 1
        if check_rule_4(seq):
            seq_to_rules[4] += 1
        if check_rule_5(seq):
            seq_to_rules[5] += 1
        if check_rule_6(seq):
            seq_to_rules[6] += 1
            
    print(seq_to_rules)
    print(N)
    return [freq / N for freq in seq_to_rules[1:]]

## Create Uniform-length Dataset: generate valid and invalid sequences

In [ ]:
from collections import defaultdict

# length of a temporal sequence
L = 20

# size of the dataset
N = 1000000

all_seqs = []
seq_to_rules = defaultdict(list)
# neg_seqs = []

use_init_token = True

for i in tqdm(range(N)):
#     seq_len = np.random.binomial(n=L, p=0.6)
    seq_len = L
    
    # Generate the time sequences only
    type_seq = [INIT_TOKEN] + np.random.randint(low=EVENT_ENCODE['A'], high=EVENT_ENCODE['D']+1, size=seq_len).tolist()
    
    # Generate a seq of timestamps. Time delta conditions on the upcoming token
    dts = []
    for et in type_seq[1:]:
        token = EVENT_TYPES[et]
        dt_dist = event_to_ts_dist[token]
        dt_sample = float(np.ceil(dt_dist()))
        dts.append(dt_sample) 
    time_seq = [0.0] + dts
        
    seq = list(zip(type_seq, time_seq))
    
    # check rules one by one:
    if check_rule_1(seq):
        seq_to_rules[i].append(1)
    if check_rule_2(seq):
        seq_to_rules[i].append(2)
    if check_rule_3(seq):
        seq_to_rules[i].append(3)
    if check_rule_4(seq):
        seq_to_rules[i].append(4)
    if check_rule_5(seq):
        seq_to_rules[i].append(5)
    if check_rule_6(seq):
        seq_to_rules[i].append(6)            
        
    all_seqs.append(seq)

In [ ]:
len(all_seqs)

In [ ]:
all_seqs[724]

## Divide pos and neg sequences by any 3 rules 

In [ ]:
pos_seqs = []
neg_seqs = []

for i in range(N):    
    if len(seq_to_rules[i]) > 3:
        pos_seqs.append(all_seqs[i])
    else:
        neg_seqs.append(all_seqs[i])

In [ ]:
print(len(pos_seqs))

In [ ]:
print(len(neg_seqs))

## Check rule distribution

In [ ]:
pos_rule_dist = get_rule_dist(pos_seqs)

In [ ]:
neg_rule_dist = get_rule_dist(neg_seqs)

In [ ]:
pos_rule_dist

In [ ]:
neg_rule_dist

## Padding and trimming

In [ ]:
def add_paddings(seq, T=21, inplace=False):
    if inplace:
        while len(seq) < T:
            seq.append((0, 0.0))
        return
    else:
        seq_copy = list(seq)
        while len(seq_copy) < T:
            seq_copy.append((0, 0.0))
        return seq_copy
    
def trim_paddings(seq, T=21, inplace=False):
    if inplace:
        while seq and seq[-1] == (0, 0.0):
            seq.pop()
        return
    else:
        seq_copy = list(seq)
        while seq_copy and seq_copy[-1] == (0, 0.0):
            seq_copy.pop()
        return seq_copy

In [ ]:
padded_pos_seqs = [add_paddings(seq) for seq in pos_seqs]
padded_neg_seqs = [add_paddings(seq) for seq in neg_seqs]
padded_all_seqs = padded_pos_seqs + padded_neg_seqs

In [ ]:
trimmed_pos_seqs = [trim_paddings(seq) for seq in padded_pos_seqs]
trimmed_neg_seqs = [trim_paddings(seq) for seq in padded_neg_seqs]

## Save Dataset : Dump into binay files 

In [ ]:
import pickle

pos_seqs_filename = 'positive_long_sequences.pickle'
neg_seqs_filename = 'negative_long_sequences.pickle'

repo_path = # '.../path-to-repo/'

with open(os.path.join(repo_path, 'data', 'long_seqs_v10', pos_seqs_filename), 'wb') as f:
    pickle.dump(pos_seqs, f)
    
with open(os.path.join(repo_path, 'data', 'long_seqs_v10', neg_seqs_filename), 'wb') as f:
    pickle.dump(neg_seqs, f)